# Derive discrete quadrant-based class labels from the raw arousal/valence annotations. Any ideas on how to aggregate conflicting annotations? Outliers?

In [267]:
import pandas as pd
import os
import numpy as np
import plotly.express as px

In [268]:
pandas_annot = pd.read_pickle(os.path.join('dataset', 'task_2_annotations_82d1d6d1093eaab6_e330cbf_pandas.pkl'))
annot_df = pd.DataFrame.from_dict(pandas_annot)
annot_df

,pianist_id,segment_id,annotator_id,arousal,valence,gems_wonder,gems_transcendence,gems_tenderness,gems_nostalgia,gems_peacefulness,gems_power,gems_joyful_activation,gems_tension,gems_sadness,gemmes_flow,gemmes_movement,gemmes_force,gemmes_interior,gemmes_wandering
0,1,0,91,1,-1,2,1,2,4,2.0,1,1,1,2,3,2,1,1.0,2
31,1,0,19,2,-1,3,3,3,4,4.0,1,2,3,3,3,2,2,3.0,3
62,1,0,189,2,0,2,1,2,1,4.0,2,2,1,1,3,2,1,1.0,4
93,1,0,126,2,2,4,5,2,3,5.0,2,4,1,3,5,1,2,2.0,5
124,1,0,26,4,2,3,5,2,3,3.0,1,3,4,1,4,1,2,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3888,11,26,36,3,-1,3,2,1,4,1.0,3,2,3,5,4,2,2,5.0,3
3919,11,26,55,2,0,2,2,2,3,3.0,3,1,3,1,2,3,2,1.0,1
3950,11,26,136,2,1,2,1,1,2,2.0,2,2,1,1,2,2,1,2.0,2
3981,11,26,169,2,0,2,1,2,2,1.0,3,4,3,2,2,3,3,1.0,1


In [269]:
pandas_features = pd.read_pickle(os.path.join('dataset', 'task_2_features_1d8b658c21ddc127_e330cbf_pandas.pkl'))
feature_df = pd.DataFrame.from_dict(pandas_features)
feature_df

,pianist_id,segment_id,snippet_id,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,...,mirtoolbox_novelty_std,mirtoolbox_novelty_pct_10,mirtoolbox_novelty_pct_50,mirtoolbox_novelty_pct_90,mirtoolbox_pulseclarity,mirtoolbox_roughness_mean,mirtoolbox_roughness_std,mirtoolbox_roughness_pct_10,mirtoolbox_roughness_pct_50,mirtoolbox_roughness_pct_90
0,1,0,0,0.192237,0.059404,2.040252,16079768.0,0.800000,0.677256,0.095846,...,0.106446,0.00000,0.013819,0.082176,0.169743,0.011449,0.004161,0.006684,0.011048,0.017025
1,1,0,1,0.143425,0.064204,3.138845,35489248.0,2.600000,0.514913,0.092372,...,0.193342,0.00000,0.027786,0.356029,0.355861,0.006205,0.003194,0.002583,0.005657,0.010610
2,1,0,2,0.146967,0.056205,2.019706,42130144.0,2.600000,0.560116,0.115359,...,0.218179,0.00000,0.049050,0.375251,0.209162,0.007467,0.003986,0.002330,0.007206,0.012690
3,1,0,3,0.158810,0.059129,3.567908,40922732.0,1.400000,0.596779,0.116061,...,0.118148,0.00002,0.017653,0.098657,0.232454,0.007335,0.004063,0.002609,0.006807,0.012971
4,1,0,4,0.168547,0.049648,2.329854,51921612.0,1.600000,0.556160,0.143420,...,0.121018,0.00000,0.020525,0.157408,0.140930,0.007345,0.003917,0.002662,0.007016,0.012731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,11,26,2,0.202177,0.050760,2.840705,51977460.0,1.800000,0.688834,0.090235,...,0.203668,0.00000,0.052074,0.377220,0.269012,0.014044,0.004991,0.007948,0.013532,0.020522
4176,11,26,3,0.174454,0.063426,5.202462,9471357.0,3.200000,0.661253,0.107627,...,0.161435,0.00000,0.029478,0.251830,0.353269,0.011126,0.004529,0.005563,0.011076,0.016763
4177,11,26,4,0.161152,0.074724,4.605277,7378860.5,2.600000,0.645111,0.093705,...,0.130765,0.00000,0.025099,0.151084,0.319188,0.009205,0.004808,0.004193,0.008083,0.016254
4178,11,26,5,0.192846,0.056795,2.563405,22641664.0,3.200000,0.680604,0.094989,...,0.247554,0.00000,0.084562,0.600001,0.133573,0.013538,0.004881,0.007919,0.012861,0.019997


In [270]:
def label_race(row):
   if row['valence'] <= 0 and row['arousal'] > 3:
        return 'angry'
   elif row['valence'] > 0 and row['arousal'] >= 3:
       return 'happy'
   elif row['valence'] < 0 and row['arousal'] <= 3:
       return 'sad'
   elif row['valence'] >= 0 and row['arousal'] < 3:
       return 'relaxed'
   else:
       return 'None'

annot_df['class'] = annot_df.apply(lambda row: label_race(row), axis=1)
annot_df

,pianist_id,segment_id,annotator_id,arousal,valence,gems_wonder,gems_transcendence,gems_tenderness,gems_nostalgia,gems_peacefulness,gems_power,gems_joyful_activation,gems_tension,gems_sadness,gemmes_flow,gemmes_movement,gemmes_force,gemmes_interior,gemmes_wandering,class
0,1,0,91,1,-1,2,1,2,4,2.0,1,1,1,2,3,2,1,1.0,2,sad
31,1,0,19,2,-1,3,3,3,4,4.0,1,2,3,3,3,2,2,3.0,3,sad
62,1,0,189,2,0,2,1,2,1,4.0,2,2,1,1,3,2,1,1.0,4,relaxed
93,1,0,126,2,2,4,5,2,3,5.0,2,4,1,3,5,1,2,2.0,5,relaxed
124,1,0,26,4,2,3,5,2,3,3.0,1,3,4,1,4,1,2,3.0,1,happy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3888,11,26,36,3,-1,3,2,1,4,1.0,3,2,3,5,4,2,2,5.0,3,sad
3919,11,26,55,2,0,2,2,2,3,3.0,3,1,3,1,2,3,2,1.0,1,relaxed
3950,11,26,136,2,1,2,1,1,2,2.0,2,2,1,1,2,2,1,2.0,2,relaxed
3981,11,26,169,2,0,2,1,2,2,1.0,3,4,3,2,2,3,3,1.0,1,relaxed


+ Are the choices of the annotators for each segment and pianist consistent?

+ What happens when we try different values to also include the None cases?

+ Maybe somehow visualize the classes... in a scatterplot with arousal and valence, colored by ... ?

In [271]:
annot_df_no_none = annot_df.copy()
annot_df_no_none['class'] = annot_df_no_none['class'].replace([None], 'None')
group_arou = annot_df_no_none.groupby(['valence', 'arousal']).count()

valance = np.repeat(np.array([-2, -1, 0, 1, 2]), 5)
arousal = np.tile(np.array([1, 2, 3, 4, 5]), 5)
scatter_data = np.stack([valance, arousal, group_arou['class']]).T

scatter_df = pd.DataFrame(data=scatter_data, columns=['valence', 'arousal', 'count'])
scatter_df['class'] = scatter_df.apply(lambda row: label_race(row), axis=1)
print(scatter_df)

    valence  arousal  count    class
0        -2        1     26      sad
1        -2        2     49      sad
2        -2        3     30      sad
3        -2        4     49    angry
4        -2        5     43    angry
5        -1        1     77      sad
6        -1        2    250      sad
7        -1        3    172      sad
8        -1        4    178    angry
9        -1        5     61    angry
10        0        1    116  relaxed
11        0        2    263  relaxed
12        0        3    191     None
13        0        4    144    angry
14        0        5     41    angry
15        1        1     60  relaxed
16        1        2    196  relaxed
17        1        3    184    happy
18        1        4    199    happy
19        1        5     70    happy
20        2        1     11  relaxed
21        2        2     46  relaxed
22        2        3     43    happy
23        2        4     80    happy
24        2        5     59    happy


In [272]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Heatmap(
                   z=np.reshape(np.array(group_arou['class']), (5, 5)).T,
                   x=['-2', '-1', '0', '1', '2'],
                   y=['1', '2', '3', '4', '5'],
                   text=np.reshape(np.array(group_arou['class']), (5, 5)).T,
                   texttemplate="%{text}",
                   textfont={"size":20}
                   ))

fig.update_xaxes(side="top")
fig.update_layout(
    title='Heatmap',
    xaxis_title="Valence",
    yaxis_title="Arousal",
    legend_title="Count",
    )

fig.show()

## Discrete quadrant based class labels

In [273]:
fig = px.scatter(scatter_df,
                 x='valence',
                 y='arousal',
                 size='count',
                 color='class',
                 width=600,
                 height=400,
                 color_discrete_map={
                "sad": px.colors.qualitative.Prism[1],
                "angry": px.colors.qualitative.Prism[7],
                "relaxed": px.colors.qualitative.Prism[4],
                "happy": px.colors.qualitative.Prism[5],
                "None": px.colors.qualitative.Prism[10]},
                title='Method 1. Discrete quadrant based class labels')
fig.show()

- what about samples with valence=0 and arousal=3 ?
- what about outliers in the data?

## Relaxing the conditions for class happy

In [274]:
scatter_df_sad = scatter_df.copy()
scatter_df_sad['class'] = scatter_df_sad['class'].replace(['None'], 'happy')
fig = px.scatter(scatter_df_sad,
                 x='valence',
                 y='arousal',
                 size='count',
                 color='class',
                 width=600,
                 height=400,
                 color_discrete_map={
                "sad": px.colors.qualitative.Prism[1],
                "angry": px.colors.qualitative.Prism[7],
                "relaxed": px.colors.qualitative.Prism[4],
                "happy": px.colors.qualitative.Prism[5],
                "None": px.colors.qualitative.Prism[10]},
                 title='Method 2. Relaxing the conditions for class happy')
fig.show()

- introduces new bias, why should all these songs be happy?
- what about outliers in the data?

## Mean value over all annotators for the same segment and pianist

In [275]:
annot_df_mean = annot_df.copy()
group_mean = annot_df_mean.groupby(['pianist_id', 'segment_id']).mean()
group_mean['class'] = group_mean.apply(lambda row: label_race(row), axis=1)

annot_df_count = annot_df.copy()
group_count = annot_df_count.groupby(['pianist_id', 'segment_id']).count()

scatter_data_mean = np.stack([group_mean['valence'], group_mean['arousal'], group_count['annotator_id']]).T

scatter_df_mean = pd.DataFrame(data=scatter_data_mean, columns=['valence', 'arousal', 'count'])
scatter_df_mean['class'] = scatter_df_mean.apply(lambda row: label_race(row), axis=1)
scatter_df_mean

,valence,arousal,count,class
0,0.181818,2.909091,11.0,relaxed
1,0.000000,3.875000,8.0,angry
2,0.181818,2.909091,11.0,relaxed
3,-0.250000,3.000000,8.0,sad
4,-0.181818,3.000000,11.0,sad
...,...,...,...,...
292,0.090909,2.181818,11.0,relaxed
293,0.000000,3.166667,6.0,angry
294,-0.181818,2.000000,11.0,sad
295,0.000000,1.833333,6.0,relaxed


In [276]:
# Get the grouped dataframe
annot_df_mean = annot_df.copy()
group_mean = annot_df_mean.groupby(['pianist_id', 'segment_id']).mean()
group_mean['class'] = group_mean.apply(lambda row: label_race(row), axis=1)

annot_df_count = annot_df.copy()
group_mean['count'] = annot_df_count.groupby(['pianist_id', 'segment_id']).count()['annotator_id']
group_mean

annotator_id   arousal   valence  gems_wonder  \
pianist_id segment_id                                                  
1          0              98.000000  2.909091  0.181818     3.272727   
           1             107.750000  3.875000  0.000000     3.375000   
           2              98.000000  2.909091  0.181818     2.818182   
           3             107.750000  3.000000 -0.250000     1.875000   
           4              98.000000  3.000000 -0.181818     2.727273   
...                             ...       ...       ...          ...   
11         22            100.090909  2.181818  0.090909     2.727273   
           23            106.833333  3.166667  0.000000     2.833333   
           24            100.090909  2.000000 -0.181818     2.454545   
           25            106.833333  1.833333  0.000000     2.833333   
           26            100.090909  2.909091  0.181818     3.090909   

                       gems_transcendence  gems_tenderness  gems_nostalgia  \
pianist_id segment_id                                                        
1          0                     3.181818         2.909091        3.727273   
           1                     3.250000         2.125000        2.375000   
           2                     2.545455         1.636364        1.545455   
           3                     1.750000         1.375000        1.750000   
           4                     2.636364         2.545455        2.545455   
...                                   ...              ...             ...   
11         22                    1.545455         3.272727        2.818182   
           23                    2.833333         1.666667        2.000000   
           24                    1.636364         3.272727        3.818182   
           25                    2.000000         1.833333        3.500000   
           26                    1.909091         1.454545        2.363636   

                       gems_peacefulness  gems_power  gems_joyful_activation  \
pianist_id segment_id                                                          
1          0                    3.454545    2.090909                2.363636   
           1                    1.750000    2.375000                2.875000   
           2                    1.545455    2.909091                3.000000   
           3                    1.375000    2.625000                1.500000   
           4                    2.272727    2.363636                1.454545   
...                                  ...         ...                     ...   
11         22                   3.454545    1.454545                2.090909   
           23                   1.333333    3.000000                1.666667   
           24                   3.636364    1.181818                1.909091   
           25                   2.833333    1.333333                1.333333   
           26                   1.818182    2.636364                2.545455   

                       gems_tension  gems_sadness  gemmes_flow  \
pianist_id segment_id                                            
1          0               2.090909      2.363636     3.818182   
           1               2.625000      1.875000     3.375000   
           2               3.363636      1.727273     3.000000   
           3               3.375000      2.375000     2.000000   
           4               2.727273      2.909091     3.000000   
...                             ...           ...          ...   
11         22              1.636364      2.545455     2.636364   
           23              2.666667      1.166667     2.833333   
           24              1.636364      2.545455     3.545455   
           25              1.500000      2.000000     3.500000   
           26              2.272727      1.727273     2.636364   

                       gemmes_movement  gemmes_force  gemmes_interior  \
pianist_id segment_id                                                   
1          0             

In [277]:
fig = px.scatter(scatter_df_mean,
                 x='valence',
                 y='arousal',
                 size='count',
                 color='class',
                 width=600,
                 height=400,
                 color_discrete_map={
                "sad": px.colors.qualitative.Prism[1],
                "angry": px.colors.qualitative.Prism[7],
                "relaxed": px.colors.qualitative.Prism[4],
                "happy": px.colors.qualitative.Prism[5],
                "None": px.colors.qualitative.Prism[10]},
                 title='Method 3. Mean value over the annotators')
fig.show()

- Good solution for assigning the ones that would have had None previously
- Also accounts for outliers

## Stacked Histogram with distribution

In [278]:
types = pd.Series(['1.', '2.', '3.']).repeat(5)
classes = pd.concat([pd.Series(['None', 'angry', 'happy', 'relaxed', 'sad'])] * 3)

scatter_df_grouped = scatter_df.groupby(['class']).sum()
scatter_df_sad_grouped = scatter_df_sad.groupby(['class']).sum()
scatter_df_mean_grouped = scatter_df_mean.groupby(['class']).sum()

counts = scatter_df_grouped['count'].tolist()
counts.append(0)
counts.append(scatter_df_sad_grouped['count'].tolist()[0])
counts.append(scatter_df_sad_grouped['count'].tolist()[1])
counts.append(scatter_df_sad_grouped['count'].tolist()[2])
counts.append(scatter_df_sad_grouped['count'].tolist()[3])
counts.append(0)
counts.append(scatter_df_mean_grouped['count'].tolist()[0])
counts.append(scatter_df_mean_grouped['count'].tolist()[1])
counts.append(scatter_df_mean_grouped['count'].tolist()[2])
counts.append(scatter_df_mean_grouped['count'].tolist()[3])

stack = np.stack([types, classes, np.array(counts)]).T
bar_df = pd.DataFrame(data=stack, columns=['methods', 'classes', 'count'])

In [279]:
fig = px.bar(bar_df,
             x="methods",
             y="count",
             color="classes",
             width=600,
             height=400,
             color_discrete_map={
             "sad": px.colors.qualitative.Prism[1],
             "angry": px.colors.qualitative.Prism[7],
             "relaxed": px.colors.qualitative.Prism[4],
             "happy": px.colors.qualitative.Prism[5],
             "None": px.colors.qualitative.Prism[10]},
             title="Class distributions",)
fig.show()